# WML Functions
In this notebook, we leverage Watson Machine Learning functions capability which enables you to create an arbitrary Python function and deploy it to scale using Watson Machine Learning service. 

In particular, we would like to create a Python function as a wrapper to a REST API endpoint for a machine learning churn prediction model which is deployed in a different environment. In the scenario we consider, the machine learning model was trained and deployed to a REST endpoint in a Cloud Pak for Data environment. All that we need is the REST endpoint so can create an adequate wrapper function that captures the input/output in a format consistent with what Watson Openscale needs for monitoring the deployed model.

In [ ]:
import os, json
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import requests

# Watson Machine Learning credentials
In the next cell, please provide the Watson Machine Learning credentials which you can get by navigating to your IBM Cloud account, finding your Watson Machine Learning service, and clicking on "Service Credentials".
```
wml_credentials = {
    "apikey": "########",
    "instance_id": "#########",
    "url": "##########"
}
```

In [ ]:

wml_credentials = {
    "apikey": "########",
    "instance_id": "############",
    "url": "##########"
}

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
client = WatsonMachineLearningAPIClient( wml_credentials )

In [ ]:
# List all the models deployed to your Watson Machine Learning instance
client.deployments.list()

# Create a deployable function

The basics of creating and deploying functions in Watson Machine Learning are given here:
- <a href="https://dataplatform.cloud.ibm.com/docs/content/analyze-data/ml-deploy-functions.html" target="_blank" rel="noopener noreferrer">Creating and deploying functions</a>
- <a href="https://dataplatform.cloud.ibm.com/docs/content/analyze-data/ml-functions.html" target="_blank" rel="noopener noreferrer">Implementation details of deployable functions</a>


### Define the function
1. Define a Python closure with an inner function named "score".
2. Use default parameters to save your Watson Machine Learning credentials and the model deployment endpoint URL with the deployed function.
3. Process the canvas data (reshape and normalize) and then send the processed data to the model deployment.
4. Process the results from the model deployment so the deployed function returns simpler results.
5. Implement error handling so the function will behave gracefully if there is an error.

Define WML deployment function so it meets the requirements of Watson Openscale for monitoring.

Specifically, Watson Openscale requires the endpoints to be compatible with the following swagger [implementation](https://aiopenscale-custom-deployement-spec.mybluemix.net/)
https://aiopenscale-custom-deployement-spec.mybluemix.net/#/Deployments/post_v1_deployments__deployment_id__online

Request Body:

```
{
  "fields": [
    "name",
    "age",
    "position"
  ],
  "values": [
    [
      "john",
      33,
      "engineer"
    ],
    [
      "mike",
      23,
      "student"
    ]
  ]
}
```

Response should match expected format for Watson OpenScale
```
{
  "fields": [
    "name",
    "age",
    "position",
    "prediction",
    "probability"
  ],
  "labels": [
    "personal",
    "camping"
  ],
  "values": [
    [
      "john",
      33,
      "engineer",
      "personal",
      [
        0.6744664422398081,
        0.32553355776019194
      ]
    ],
    [
      "mike",
      23,
      "student",
      "camping",
      [
        0.2794765664946941,
        0.7205234335053059
      ]
    ]
  ]
}
```


**In the next section, replace the cp4d_scoring_url and Authorization token with the correct valuess.**
- **cp4d_scoring_url**: The cp4d_scoring_url is the REST endpoint for the machine learning model you had trained and deployed in the other environment (In our case, the Cloud Pak for Data cluster). 
- **auth_token**: The auth_token is the authroization token generated for accessing the REST endpoint of the trained model in the other environment



In [ ]:
ai_parms = {}

def churn_deployable_function( parms=ai_parms ):
    
    def score( function_payload ):
            
        cp4d_scoring_url = '#######'
        auth_token = '########'

        try:
            import requests
            import json
            
            url = cp4d_scoring_url
            headers = {
                'Cache-Control': 'no-cache',
                'Authorization': auth_token,
                'Content-Type': 'application/json',
            }
       
    
            fields = function_payload['fields']
            values = function_payload['values']
            
            # Update the fields with prediction and probability entries
            outputfields = fields.copy()
            outputfields.extend(['prediction','probability'])
            
            
            alloutputvalues = []
            for index in range(0,len(values)):
                data = json.dumps({"args":{"input_json":[dict(zip(fields, values[index]))]}})
                
                print("data: ", data)
                ## This one works fine (note the verify=False parm)
                response = requests.post(url, headers=headers, data=data, verify=False)
                
                # Map response into a json dict
                response_json = json.loads(response.text)
                
                # Make a copy of values list
                outputvalues = values[index].copy()
                
                # Extract the prediction from the response
                prediction = response_json['result']['predictions'][0]
                
                # Extract the probabilities form the response
                # Note that the probabilities should match the same order of the classes (or labels)
                probabilities = response_json['result']['probabilities'][0]
                
                # Update the values entry with the values for prediction and probabilities
                outputvalues.extend([prediction,probabilities])
                
                alloutputvalues.append(outputvalues)
                
                # Extract the labels
                labels = response_json['result']['classes']
                
            final_response = {"fields":outputfields, "labels":labels, "values":alloutputvalues}
            
            return final_response
            
        except Exception as e:
            
            return { "error" : repr( e ) }


    return score

### Test the Function locally
You can test your function in the notebook before deploying the function.

In [ ]:
fields = ['LONGDISTANCE', 'INTERNATIONAL', 'LOCAL', 'DROPPED', 'PAYMETHOD', 'LOCALBILLTYPE', 'LONGDISTANCEBILLTYPE', 'USAGE', 'RATEPLAN', 'GENDER', 'STATUS', 'CHILDREN', 'ESTINCOME', 'CAROWNER', 'AGE']
values = [
    [23, 0, 206, 0, 'CC', 'Budget', 'Intnl_discount', 229, 3, 'F', 'S', 1, 38000.0, 'N', 24.393]
]
test_data = {
    "fields": fields,
    "values": values
}

In [ ]:
# Pass the sample canvas data to the function as a test
func_result = churn_deployable_function()(test_data)
print( func_result )

### Store and deploy the function to WML
Before you can deploy the function, you must store the function in your Watson Machine Learning repository.

Provide a unique name for the deployed function in WML so you can identify and retrieve efficiently.

In [ ]:
# Store the deployable function in your Watson Machine Learning repository
# 
meta_data = { client.repository.FunctionMetaNames.NAME : 'Customer Churn ICP4D labuser1' }
function_details = client.repository.store_function( meta_props=meta_data, function=churn_deployable_function )

In [ ]:
# Deploy the stored function
#
function_id = function_details["metadata"]["guid"]
function_deployment_details = client.deployments.create( artifact_uid=function_id, name="Customer Churn ICP4D labuser1" )

In [ ]:
# Verify function shows up in the deployments for WML
# In the listing below, you should see the Name for the model you deployed.
client.deployments.list()

### Test the deployed function in WML
You can use the Watson Machine Learning Python client or REST API to send data to your function deployment for processing in exactly the same way you send data to model deployments for processing.

In the listing above, you should see the Name of the model you deployed. For that model, extract the deployment_id (first column from list above) and specify it in the next cell.

In [ ]:
deployment_id = '############'

In [ ]:
function_deployment_details = client.deployments.get_details(deployment_id)

In [ ]:
# Get the endpoint URL of the function deployment just created
#
function_deployment_endpoint_url = client.deployments.get_scoring_url( function_deployment_details )
function_deployment_endpoint_url

In [ ]:
# Test the WML deployed function
result = client.deployments.score( function_deployment_endpoint_url, test_data )
if "error" in result:
    print( result["error"] )
else:
    print( result )

# Conclusion
With this notebook, we created a function wrapper to a REST endpoint of a trained machine learning model which is deployed in a different environment and deployed it to Watson Machine Learning on IBM Cloud. This allows us to monitor this machine learning model for accuracy, fairness, drift, and explainability with Watson Openscale.